In [1]:
import numpy as np
from scipy import sparse
import operator

In [3]:
def build_sparse_link_matrix(filename):
    """
    Constructs the Link Matrix A in Sparse Format (CSR).
    
    CRITICAL OPTIMIZATION:
    This function does NOT physically apply the 'dangling node patch' (filling 
    columns with 1/N). Doing so would destroy sparsity and consume O(N^2) memory.
    Instead, it identifies dangling nodes so we can handle them 'virtually' 
    during the calculation phase.
    
    Args:
        filename (str): Path to the .dat file.
        
    Returns:
        tuple: (A_sparse, N, dangling_indices)
    """
    links = []
    out_degree = {}
    
    # 1) READING THE DATASET FILE
    print(f"Reading file: {filename}...")
    try:
        with open(filename, 'r') as file:
            # 1. Read Header (N = Nodes, M = Edges)
            header = file.readline().strip().split()
            if not header: raise ValueError("File is empty or header is missing.")
            N = int(header[0])
            
            # 2. Skip URL mapping lines (we only need the graph structure)
            # The first N lines are strings/URLs which we don't need for the math.
            for _ in range(N): file.readline()
            
            # 3. Read Edges
            # Format: "Source_ID Target_ID"
            for line in file:
                parts = line.strip().split()
                if len(parts) == 2:
                    src, tgt = int(parts[0]), int(parts[1])
                    
                    out_degree[src] = out_degree.get(src, 0) + 1
                    links.append((src, tgt))
                    
    except Exception as e:
        print(f"Error reading file: {e}")
        return None, 0, None

    # 2) SPARSE MATRIX CONSTRUCTION ---
    # We use the COO format logic initially: separate lists for Data, Rows, Cols
    data = []
    rows = [] # Target Node (i)
    cols = [] # Source Node (j)
    
    # Identify Dangling Nodes (Nodes with 0 out-degree)
    # Initialize a boolean mask: assume ALL are dangling initially.
    is_dangling = np.ones(N, dtype=bool) 
    
    for src, tgt in links:
        # Adjust 1-based IDs from file to 0-based Indices for Python
        src_idx = src - 1
        tgt_idx = tgt - 1
        
        # If a node appears as a source, it has outgoing links -> Not Dangling
        is_dangling[src_idx] = False
        
        # Calculate transition probability: 1 / (Number of outgoing links)
        # This makes valid columns sum to 1.
        val = 1.0 / out_degree[src]
        
        rows.append(tgt_idx)
        cols.append(src_idx)
        data.append(val)
        
    # Convert to CSR format.
    # CSR is extremely efficient for Matrix-Vector multiplication (A @ x).
    A_sparse = sparse.csr_matrix((data, (rows, cols)), shape=(N, N))
    
    # Extract indices of dangling nodes for later use
    dangling_indices = np.where(is_dangling)[0]
    
    print(f"Sparse Matrix Constructed: {N} nodes.")
    print(f"Valid Non-Zero Links: {len(data)}.")
    print(f"Dangling Nodes Identified: {len(dangling_indices)}.")
    
    return A_sparse, N, dangling_indices

In [4]:
def calculate_pagerank(A_sparse, N, dangling_indices, m, max_iter=200, tol=1e-7):
    """
    Computes PageRank using the Power Method with Sparse Optimizations.
    
    Mathematical Logic:
    x_new = (1-m)*Ax + (1-m)*[Dangling_Correction] + m/N
    
    Args:
        A_sparse: The sparse link matrix (missing dangling connections).
        N: Total nodes.
        dangling_indices: List of nodes that have no outgoing links.
        m: Teleportation probability (1-m is the damping factor).
        
    Returns:
        tuple: (PageRank Vector, Iterations)
    """
    
    # 1. Initialization
    # Start with uniform probability distribution (1/N for everyone)
    x = np.full(N, 1.0/N)
    
    # 2. Teleportation Constant
    # This is the "m * s" part of the formula.
    # Since s is [1/N, 1/N...], this term is just the scalar m/N added to every node.
    teleport_contribution = m / N
    
    # Ensure dangling_indices is a numpy array for fast indexing
    if not isinstance(dangling_indices, np.ndarray):
        dangling_indices = np.array(dangling_indices)
        
    iterations = 0
    
    # --- POWER METHOD LOOP ---
    for k in range(max_iter):
        x_prev = x.copy()
        
        # Step A: Standard Matrix Multiplication
        # This calculates flow only from nodes that have existing links.
        Ax = A_sparse.dot(x_prev)
        
        # Step B: Implicit Dangling Node Handling
        # Total mass from dangling nodes lost.
        dangling_mass_sum = np.sum(x_prev[dangling_indices])
        
        # We redistribute this lost mass evenly to ALL nodes (1/N), 
        # applied with the damping factor (1-m).
        dangling_correction = (1 - m) * (dangling_mass_sum / N)
        
        # Step C: Combine Everything
        # 1. Flow from Links
        # 2. Flow from Dangling Patch
        # 3. Flow from Random Teleport
        x = (1 - m) * Ax + dangling_correction + teleport_contribution
        
        # Step D: Convergence Check (L1 Norm)
        diff = np.sum(np.abs(x - x_prev))
        iterations = k + 1
        
        if diff < tol:
            break
            
    return x, iterations

In [5]:
# Test on the required graphs.
print("=============================================")
print("  EXECUTION: WEB WITH 4 PAGES (FIGURE 2.1) ")
print("=============================================")

# Construction of the link matrix
A_4pages = np.array([
    [0.0, 0.0, 1.0, 0.5],
    [1/3, 0.0, 0.0, 0.0],
    [1/3, 0.5, 0.0, 0.5],
    [1/3, 0.5, 0.0, 0.0]
])

# Calculation of PageRank
pagerank_scores_4pages, iterations_4pages = calculate_pagerank(A_4pages, A_4pages.shape[0], False, m=0.15)

# Preparation of results: list of tuples (Page ID, Score)
page_indices_4pages = np.arange(1, A_4pages.shape[0] + 1)
results_4pages = list(zip(page_indices_4pages, pagerank_scores_4pages.flatten()))
results_4pages_sorted = sorted(results_4pages, key=operator.itemgetter(1), reverse=True)

# Printing the results
for page_id, score in results_4pages_sorted:
    print(f"Page {page_id}: {score:.4f}")
    
print(f"Calculation completed in {iterations_4pages} iterations.")


print("\n\n=============================================")
print("  EXECUTION: WEB WITH 5 PAGES (FIGURE 2.2) ")
print("=============================================")

#Construction of the link matrix
A_5pages = np.array([
    [0.0, 1.0, 0.0, 0.0, 0.0],   
    [1.0, 0.0, 0.0, 0.0, 0.0],  
    [0.0, 0.0, 0.0, 1.0, 0.5],   
    [0.0, 0.0, 1.0, 0.0, 0.5],   
    [0.0, 0.0, 0.0, 0.0, 0.0]    
])

# Calculation of PageRank
pagerank_scores_5pages,iterations_5pages = calculate_pagerank(A_5pages, A_5pages.shape[0], False, m=0.15)

# Preparation of results: list of tuples (Page ID, Score)
page_indices_5pages = np.arange(1, A_5pages.shape[0] + 1)
results_5pages = list(zip(page_indices_5pages, pagerank_scores_5pages.flatten()))
results_5pages_sorted = sorted(results_5pages, key=operator.itemgetter(1), reverse=True)

# Printing the results
for page_id, score in results_5pages_sorted:
    print(f"Page {page_id}: {score:.4f}")
    
print(f"Calculation completed in {iterations_5pages} iterations.")

  EXECUTION: WEB WITH 4 PAGES (FIGURE 2.1) 
Page 1: 0.3682
Page 3: 0.2880
Page 4: 0.2021
Page 2: 0.1418
Calculation completed in 21 iterations.


  EXECUTION: WEB WITH 5 PAGES (FIGURE 2.2) 
Page 3: 0.2850
Page 4: 0.2850
Page 1: 0.2000
Page 2: 0.2000
Page 5: 0.0300
Calculation completed in 2 iterations.


In [7]:
filename = 'hollins.dat'
m = 0.15      # Standard Google damping factor
top_k = 10    # Number of top results to display

# 1. Build the Matrix
A_sparse, N, dangling_nodes = build_sparse_link_matrix(filename)

# 2. Calculate PageRank
if A_sparse is not None:
    pagerank_scores, iters = calculate_pagerank(A_sparse, N, dangling_nodes, m=m)

    print(f"Calculation converged in {iters} iterations.")

    # 3. Display Results
    # Create pairs of (Page ID, Score)
    page_ids = np.arange(1, N + 1)
    results = list(zip(page_ids, pagerank_scores))
    
    # Sort by score descending
    results_sorted = sorted(results, key=operator.itemgetter(1), reverse=True)

    print(f"\n--- TOP {top_k} RANKING ---")
    print(f"{'Rank':<5} | {'Page ID':<10} | {'Score':<15}")
    print("-" * 35)
    
    for rank, (page_id, score) in enumerate(results_sorted[:top_k], 1):
        print(f"{rank:<5} | {page_id:<10} | {score:.6f}")

    # 4. Mathematical Verification
    total_prob = np.sum(pagerank_scores)
    print("\n--- VERIFICATION ---")
    print(f"Total Probability Sum: {total_prob:.6f} (Should be 1.0)")
    
    # Minimum Score Check
    min_score = results_sorted[-1][1]
    expected_min = m / N
    print(f"Lowest PageRank Score:   {min_score:.8f}")
    print(f"Theoretical Minimum (m/N): {expected_min:.8f}")

Reading file: hollins.dat...
Sparse Matrix Constructed: 6012 nodes.
Valid Non-Zero Links: 23875.
Dangling Nodes Identified: 3189.
Calculation converged in 71 iterations.

--- TOP 10 RANKING ---
Rank  | Page ID    | Score          
-----------------------------------
1     | 2          | 0.019879
2     | 37         | 0.009288
3     | 38         | 0.008610
4     | 61         | 0.008065
5     | 52         | 0.008027
6     | 43         | 0.007165
7     | 425        | 0.006583
8     | 27         | 0.005989
9     | 28         | 0.005572
10    | 4023       | 0.004452

--- VERIFICATION ---
Total Probability Sum: 1.000000 (Should be 1.0)
Lowest PageRank Score:   0.00005806
Theoretical Minimum (m/N): 0.00002495


In [8]:
print("=============================================")
print("                 EXERCISE 1")
print("=============================================")

# ---------------------------------------------------------
# 1. Original Situation (4 Pages)
# ---------------------------------------------------------
print("\n--- 1. Original Situation (4 Pages) ---")

A_orig_dense = np.array([
    [0.0, 0.0, 1.0, 0.5],  
    [1/3, 0.0, 0.0, 0.0],  
    [1/3, 0.5, 0.0, 0.5],  
    [1/3, 0.5, 0.0, 0.0]   
])

# Convert to sparse and calculate
# We pass [] as dangling_indices because this matrix is strictly column-stochastic (cols sum to 1)
A_orig_sparse = sparse.csr_matrix(A_orig_dense)
scores_orig, iters_orig = calculate_pagerank(A_orig_sparse,4,False,m=0.0)

# Print Original Results
indices_orig = np.arange(1, 5)
results_orig = list(zip(indices_orig, scores_orig))
results_orig_sorted = sorted(results_orig, key=operator.itemgetter(1), reverse=True)

print("Original Ranking:")
for page_id, score in results_orig_sorted:
    print(f"Page {page_id}: {score:.4f}")

print(f"Calculation completed in {iters_orig} iterations.")

# FIX: Access as 1D array
p1_score_orig = scores_orig[0]
p3_score_orig = scores_orig[2]


# ---------------------------------------------------------
# 2. Modified Situation (5 Pages)
# ---------------------------------------------------------
print("\n--- 2. Modified Situation (5 Pages) ---")
print("Modification: Added Page 5. Links: 3->5 and 5->3.")

# Modified A Matrix (5x5)
A_mod_dense = np.array([
    [0.0, 0.0, 0.5, 0.5, 0.0],  
    [1/3, 0.0, 0.0, 0.0, 0.0],  
    [1/3, 0.5, 0.0, 0.5, 1.0],  
    [1/3, 0.5, 0.0, 0.0, 0.0],  
    [0.0, 0.0, 0.5, 0.0, 0.0]   
])

A_mod_sparse = sparse.csr_matrix(A_mod_dense)
scores_mod, iters_mod = calculate_pagerank(A_mod_sparse,5,False,m=0.0)

# Print Modified Results
indices_mod = np.arange(1, 6)
results_mod = list(zip(indices_mod, scores_mod))
results_mod_sorted = sorted(results_mod, key=operator.itemgetter(1), reverse=True)


for page_id, score in results_mod_sorted:
    print(f"Page {page_id}: {score:.4f}")

print(f"Calculation completed in {iters_mod} iterations.")

# Access as 1D array
p1_score_mod = scores_mod[0]
p3_score_mod = scores_mod[2]

# ---------------------------------------------------------
# FINAL CHECK
# ---------------------------------------------------------
print("\n=============================================")
print("               RESULTS CHECK")
print("=============================================")
print(f"BEFORE: Page 1 ({p1_score_orig:.4f}) vs Page 3 ({p3_score_orig:.4f})")
print(f"AFTER:  Page 1 ({p1_score_mod:.4f}) vs Page 3 ({p3_score_mod:.4f})")

if p3_score_mod > p1_score_mod:
    print("\nANSWER: YES. The strategy worked.")
else:
    print("\nANSWER: NO.")

                 EXERCISE 1

--- 1. Original Situation (4 Pages) ---
Original Ranking:
Page 1: 0.3871
Page 3: 0.2903
Page 4: 0.1935
Page 2: 0.1290
Calculation completed in 27 iterations.

--- 2. Modified Situation (5 Pages) ---
Modification: Added Page 5. Links: 3->5 and 5->3.
Page 3: 0.3673
Page 1: 0.2449
Page 5: 0.1837
Page 4: 0.1224
Page 2: 0.0816
Calculation completed in 49 iterations.

               RESULTS CHECK
BEFORE: Page 1 (0.3871) vs Page 3 (0.2903)
AFTER:  Page 1 (0.2449) vs Page 3 (0.3673)

ANSWER: YES. The strategy worked.


In [9]:
print("=============================================")
print(" EXERCISE 11: PAGERANK CALCULATION ON MODIFIED NETWORK")
print(" (Eigenvector of M with m=0.15)")
print("=============================================")

A_ex11_dense = np.array([
    [0.0, 0.0, 0.5, 0.5, 0.0],  
    [1/3, 0.0, 0.0, 0.0, 0.0],  
    [1/3, 0.5, 0.0, 0.5, 1.0],  
    [1/3, 0.5, 0.0, 0.0, 0.0],  
    [0.0, 0.0, 0.5, 0.0, 0.0]   
])

A_ex11_sparse = sparse.csr_matrix(A_ex11_dense)
scores_ex11, iters_ex11 = calculate_pagerank(A_ex11_sparse, 5, False, m=0.15)

print(f"\nCalculation converged in {iters_ex11} iterations.")
print("Final Ranking:")

indices_ex11 = np.arange(1, 6)
results_ex11 = list(zip(indices_ex11, scores_ex11))
results_ex11_sorted = sorted(results_ex11, key=operator.itemgetter(1), reverse=True)

for page_id, score in results_ex11_sorted:
    print(f"Page {page_id}: {score:.4f}")

p1 = scores_ex11[0]
p3 = scores_ex11[2]
print(f"\nKey Comparison:")
print(f"Page 3 ({p3:.4f}) > Page 1 ({p1:.4f})?")
print("YES" if p3 > p1 else "NO")

 EXERCISE 11: PAGERANK CALCULATION ON MODIFIED NETWORK
 (Eigenvector of M with m=0.15)

Calculation converged in 33 iterations.
Final Ranking:
Page 3: 0.3489
Page 1: 0.2371
Page 5: 0.1783
Page 4: 0.1385
Page 2: 0.0972

Key Comparison:
Page 3 (0.3489) > Page 1 (0.2371)?
YES
